# COGS 108 - Data Checkpoint

# Names

- Cameron VanderTuig
- Wilson Tan
- Erdogan Ergit
- Tony Diep
- Henry Chan

<a id='research_question'></a>
# Research Question

How does the release of highly expected, popular, and graphically intensive games affect CPU and GPU sales globally?

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name: Steam Hardware & Software Survey
- Link to the dataset: https://web.archive.org/web/20170801231343/http://store.steampowered.com/hwsurvey/videocard
- Number of observations: 211
- This dataset is a survey that steam users can opt into to measure which hardware they have.
---
- Dataset Name: Benchmarking Data
- Link to the dataset: https://benchmarks.ul.com/hardware/gpu/
- Number of observations: 211
- Contains the relative performance statistics for many graphics hardware systems. It gives us a way to equate a GPU to the performance it is capable of.
---
- Dataset Name: Game_Main
- Link to the dataset: https://www.kaggle.com/deepann/80000-steam-games-dataset/version/1
- Number of observations: 81048
- This dataset is a compiled set of scrapable steam game data. This particular set contains review statistics, game names, release dates, and urls to the steam page.
---
- Dataset Name: Game_Requirements
- Link to the dataset: https://www.kaggle.com/deepann/80000-steam-games-dataset/version/1
- Number of observations: 81048
- This is the second dataset of steam games. It contains the system requirements scraped from the games' steam pages.
---
We ultimately want to know how releases of videogames affect sales of computer hardware, so we need an avenue to get from video games and their popularity to computer hardware sales. The way we anticipate doing that now is like so:
Video game popularity and release date (Game_Main) --> the video game's hardware requirements (Game_Requirements) --> the hardware in those requirements (Game_Requirements) --> the level of graphical intensity (Benchmarking Data) --> hardware that can handle that intensity (Benchmarking Data) --> sales and release date of hardware (Steam Hardware & Software Survey).
And vice versa.

# Setup

In [1]:
import requests
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

In this first section is a script that scrapes internet archive for steam survey data for any arbitrary month that we need during analysis, as well as scraping benchmarks.ul for performance data.

In [2]:
# Get URL. URL will vary
URL = "https://web.archive.org/web/20170801231343/http://store.steampowered.com/hwsurvey/videocard"
page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")

# Get names and increase/decrease percentages
name = soup.find_all("div", {"class", "substats_col_left"})
stat = soup.find_all("span", {"class", "stat_increase", "stat_decrease", "stat_unchanged"})

names = []
stats = []
performances = []
directx_version = []

for tag in stat:
	stats.append(tag.text.strip())

for x in range(1, 7):
	stats.pop(0)

temp = ""
unwanted = ["overall distribution of cards", "directx 11 gpus", "directx 10 gpus", "directx 9 shader model 2b and 3.0 gpus",
"directx 9 shader model 2.0 gpus", "directx 8 gpus and below", "all video cards"]

# "directx 12 gpus"

directx = 0

# Append only the wanted names
for tag in name:
	temp = tag.text.strip()
	if (temp.lower() in unwanted):
		directx = 0
		continue

	# Limit to most recent DirectX version 
	if (temp.lower() == "directx 12 gpus"):
		directx = 12
		continue

	directx_version.append(directx)
	names.append(temp)

page.close()

In [3]:
card = ""
# Get performance for all cards
for name in names:

	# Save some time by removing search of unwanted GPUs
	if (directx_version[names.index(name)] == 0):
		performances.append(0)
		continue

	card = name.replace(" ", "+")

	URL = "https://benchmarks.ul.com/hardware/gpu/" + card + "+review"
	page = requests.get(URL)

	soup = BeautifulSoup(page.content, "html.parser")

	try:
		performance = soup.find_all("span", {"class", "result-pimp-badge-score-item"})[0].text.strip()
		performances.append(performance)
	except:
		performances.append(0)

	page.close()

In [4]:
# Create dataframe (GPU)
df = pd.DataFrame({"GPU Name": names, "Change (%)": stats, "Performance (3DMax)": performances, "DirectX Version": directx_version})

df

,GPU Name,Change (%),Performance (3DMax),DirectX Version
0,Other,0.00%,0,0
1,NVIDIA GeForce GTX 1060,+0.97%,3740,12
2,NVIDIA GeForce GTX 750 Ti,-0.04%,1285,12
3,NVIDIA GeForce GTX 960,+0.69%,2308,12
4,NVIDIA GeForce GTX 970,+0.01%,3659,12
...,...,...,...,...
206,Intel Q965/Q963 Express,+0.16%,0,0
207,ATI Radeon 9500/9700,+1.19%,0,0
208,ATI Mobility Radeon X300,-0.18%,0,0
209,VIA Chrome9 HC IGP,-0.17%,0,0


In [5]:
# Create dataframe (Game name)
df_game_main = pd.read_csv('Game_Main.csv')
df_game_main

,url,name,all_reviews,date
0,https://store.steampowered.com/app/945360/Amon...,Among Us,"Overwhelmingly Positive(224,878)- 95% of the 2...",16-Nov-18
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive,"Very Positive(4,843,904)- 87% of the 4,843,904...",21-Aug-12
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,"Very Positive(223,706)- 80% of the 223,706 use...",3-Aug-20
3,https://store.steampowered.com/app/1158310/Cru...,Crusader Kings III,"Very Positive(18,951)- 92% of the 18,951 user ...",1-Sep-20
4,https://store.steampowered.com/app/1085660/Des...,Destiny 2,"Very Positive(284,689)- 86% of the 284,689 use...",1-Oct-19
...,...,...,...,...
81043,https://store.steampowered.com/bundle/2961/Ste...,-,-,-
81044,https://store.steampowered.com/bundle/3123/Det...,-,-,-
81045,https://store.steampowered.com/bundle/3175/Fea...,-,-,-
81046,https://store.steampowered.com/bundle/3176/Fea...,-,-,-


In [6]:
# Create dataframe (Game name)
df_game_requirements = pd.read_csv('Game_Requirements.csv')
df_game_requirements

,url,requirements
0,https://store.steampowered.com/app/945360/Amon...,System RequirementsMinimum:OS: Windows 7 SP1+P...
1,https://store.steampowered.com/app/730/Counter...,System RequirementsWindowsMac OS XSteamOS + Li...
2,https://store.steampowered.com/app/1097150/Fal...,System RequirementsMinimum:Requires a 64-bit p...
3,https://store.steampowered.com/app/1158310/Cru...,System RequirementsWindowsMac OS XSteamOS + Li...
4,https://store.steampowered.com/app/1085660/Des...,System RequirementsMinimum:Requires a 64-bit p...
...,...,...
81043,https://store.steampowered.com/bundle/2961/Ste...,-
81044,https://store.steampowered.com/bundle/3123/Det...,-
81045,https://store.steampowered.com/bundle/3175/Fea...,-
81046,https://store.steampowered.com/bundle/3176/Fea...,-


# Data Cleaning

In our analysis, we plan to look at latest GPUs. This is because newer GPUs have higher performances and newer, graphically intensive games require higher performances.

Removing unwanted GPUs

The data originally consisted with older GPUs. During setup, these GPUs were labelled with a "DirectX Version" of 0 for faster setup. We can use this again to get rid of the older GPUs.

In [7]:
# Clean data
# Remove unwanted GPUs
df = df[df["DirectX Version"] != 0]
df = df[df["GPU Name"] != "Other"]

Cleaning dtypes

When web scrapping, the data collected wasn't in the correct type. Two columns, "Change %" and "Performance (3DMax)" columns, were incorrect.

Performance (3DMax) column
The entries in this column were Strings and needed to be converted to integers. Using the int() function to convert to integers was enough.

Change % column
Entries in this column contained a "%" symbol and couldn't be converted to the correct dtype, float, as easily. First the "%" character needed to be removed to then use the float() function.

In [8]:
# Clean dtypes
df["Change (%)"] = df["Change (%)"].apply(lambda x: float(x.replace("%","")))
df["Performance (3DMax)"] = df["Performance (3DMax)"].apply(lambda x: int(x))

Filling in missing values

Several GPUs had missing performances. To fix this, the average of the other performances were used instead.

In [9]:
# Use mean to fill in 0 values
temp_df = df[df["DirectX Version"] == 12]
temp_df = temp_df[temp_df["Performance (3DMax)"] != 0]

mean_12 = int(temp_df["Performance (3DMax)"].mean())

df["Performance (3DMax)"] = df["Performance (3DMax)"].replace(0, mean_12)

Getting higher performing GPUs

To base what are "higher performing" GPUs, the 75th percentile was calculated. Then GPUs with performance scores higher than the 75th percentile were taken.

In [10]:
# Get higher performing cards only. Based on 75th percentile
percentile = df["Performance (3DMax)"].quantile([0.75]).loc[0.75]
df = df.loc[df["Performance (3DMax)"] > percentile]
df = df.set_index(np.array(range(0, df.shape[0])))

This leaves us with only the higher performing (latest) GPUs.

In [11]:
df

,GPU Name,Change (%),Performance (3DMax),DirectX Version
0,NVIDIA GeForce GTX 1060,0.97,3740,12
1,NVIDIA GeForce GTX 960,0.69,2308,12
2,NVIDIA GeForce GTX 970,0.01,3659,12
3,NVIDIA GeForce GTX 1070,0.25,6083,12
4,NVIDIA GeForce GTX 1050 Ti,0.69,2356,12
5,NVIDIA GeForce GTX 1080,0.06,7584,12
6,AMD Radeon RX 480,-0.03,4087,12
7,NVIDIA GeForce GTX 980,-0.02,4383,12
8,NVIDIA GeForce GTX 770,-0.03,2158,12
9,NVIDIA GeForce GTX 980 Ti,-0.03,5802,12


We can combine df1 and df2 together

In [12]:
df_game = pd.merge(df_game_main, df_game_requirements)

We then drop any empty row ( == '-')

In [13]:
df_game.drop(df_game.loc[df_game['url'] == "-"].index, inplace=True)
df_game.drop(df_game.loc[df_game['name'] == "-"].index, inplace=True)
df_game.drop(df_game.loc[df_game['all_reviews'] == "-"].index, inplace=True)
df_game.drop(df_game.loc[df_game['date'] == "-"].index, inplace=True)
df_game.drop(df_game.loc[df_game['requirements'] == "-"].index, inplace=True)

df_game

,url,name,all_reviews,date,requirements
0,https://store.steampowered.com/app/945360/Amon...,Among Us,"Overwhelmingly Positive(224,878)- 95% of the 2...",16-Nov-18,System RequirementsMinimum:OS: Windows 7 SP1+P...
1,https://store.steampowered.com/app/945360/Amon...,Among Us,"Overwhelmingly Positive(224,878)- 95% of the 2...",16-Nov-18,System RequirementsMinimum:OS: Windows 7 SP1+P...
2,https://store.steampowered.com/app/945360/Amon...,Among Us,"Overwhelmingly Positive(224,878)- 95% of the 2...",16-Nov-18,System RequirementsMinimum:OS: Windows 7 SP1+P...
3,https://store.steampowered.com/app/945360/Amon...,Among Us,"Overwhelmingly Positive(224,878)- 95% of the 2...",16-Nov-18,System RequirementsMinimum:OS: Windows 7 SP1+P...
4,https://store.steampowered.com/app/945360/Amon...,Among Us,"Overwhelmingly Positive(224,878)- 95% of the 2...",16-Nov-18,System RequirementsMinimum:OS: Windows 7 SP1+P...
...,...,...,...,...,...
81379,https://store.steampowered.com/app/1346541/STA...,STAR WARS™: The Old Republic™ - Sith Bundles,"21 Jul, 2020","21 Jul, 2020",System RequirementsMinimum:OS: Windows 7 or la...
81380,https://store.steampowered.com/app/1347970/Pat...,Patch Quest,Late 2020,Late 2020,System RequirementsMinimum:OS: Windows 10Proce...
81381,https://store.steampowered.com/app/1349120/_/?...,球球少女,"6 Nov, 2020","6 Nov, 2020",System RequirementsMinimum:OS: WIN7 SP1/WIN8/W...
81382,https://store.steampowered.com/app/1349170/Fur...,Furries & Scalies & Bears OH MY! 2: Return to ...,"20 Apr, 2021","20 Apr, 2021",System RequirementsWindowsSteamOS + LinuxMinim...


Remove duplicates

In [14]:
df_game.drop_duplicates(inplace=True, keep="first")
df_game = df_game.dropna()
df_game.set_index(np.array(range(0, df_game.shape[0])), inplace=True)
df_game

,url,name,all_reviews,date,requirements
0,https://store.steampowered.com/app/945360/Amon...,Among Us,"Overwhelmingly Positive(224,878)- 95% of the 2...",16-Nov-18,System RequirementsMinimum:OS: Windows 7 SP1+P...
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive,"Very Positive(4,843,904)- 87% of the 4,843,904...",21-Aug-12,System RequirementsWindowsMac OS XSteamOS + Li...
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,"Very Positive(223,706)- 80% of the 223,706 use...",3-Aug-20,System RequirementsMinimum:Requires a 64-bit p...
3,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,"Very Positive(223,725)- 80% of the 223,725 use...",3-Aug-20,System RequirementsMinimum:Requires a 64-bit p...
4,https://store.steampowered.com/app/1158310/Cru...,Crusader Kings III,"Very Positive(18,951)- 92% of the 18,951 user ...",1-Sep-20,System RequirementsWindowsMac OS XSteamOS + Li...
...,...,...,...,...,...
74220,https://store.steampowered.com/app/1346541/STA...,STAR WARS™: The Old Republic™ - Sith Bundles,"21 Jul, 2020","21 Jul, 2020",System RequirementsMinimum:OS: Windows 7 or la...
74221,https://store.steampowered.com/app/1347970/Pat...,Patch Quest,Late 2020,Late 2020,System RequirementsMinimum:OS: Windows 10Proce...
74222,https://store.steampowered.com/app/1349120/_/?...,球球少女,"6 Nov, 2020","6 Nov, 2020",System RequirementsMinimum:OS: WIN7 SP1/WIN8/W...
74223,https://store.steampowered.com/app/1349170/Fur...,Furries & Scalies & Bears OH MY! 2: Return to ...,"20 Apr, 2021","20 Apr, 2021",System RequirementsWindowsSteamOS + LinuxMinim...


Reformatting the reviews

We're only interested in number of reviews and percentage of likes. We can use regulat expressions and the apply function to get rid of useless information.

In [15]:
df_game["all_reviews"] = df_game["all_reviews"].apply(lambda x: x.lower())
df_game["all_reviews"] = df_game["all_reviews"].apply(lambda x: x.strip())
df_game["all_reviews"] = df_game["all_reviews"].apply(lambda x: re.sub("[^0-9\s]", "", x))
df_game["all_reviews"]

<ipython-input-15-b509d8a4284a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_game["all_reviews"] = df_game["all_reviews"].apply(lambda x: x.lower())
<ipython-input-15-b509d8a4284a>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_game["all_reviews"] = df_game["all_reviews"].apply(lambda x: x.strip())
<ipython-input-15-b509d8a4284a>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

0           224878 95   224878       
1         4843904 87   4843904       
2           223706 80   223706       
3           223725 80   223725       
4             18951 92   18951       
                     ...             
74220                        21  2020
74221                            2020
74222                         6  2020
74223                        20  2021
74224                        12  2017
Name: all_reviews, Length: 74225, dtype: object

Since not all games had useable "all_reviews" data, we can get rid of those rows.

In [16]:
df_game = df_game[df_game["all_reviews"].str.split().apply(len) == 3]
df_game.set_index(np.array(range(0, df_game.shape[0])), inplace=True)
df_game

,url,name,all_reviews,date,requirements
0,https://store.steampowered.com/app/945360/Amon...,Among Us,224878 95 224878,16-Nov-18,System RequirementsMinimum:OS: Windows 7 SP1+P...
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive,4843904 87 4843904,21-Aug-12,System RequirementsWindowsMac OS XSteamOS + Li...
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,223706 80 223706,3-Aug-20,System RequirementsMinimum:Requires a 64-bit p...
3,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,223725 80 223725,3-Aug-20,System RequirementsMinimum:Requires a 64-bit p...
4,https://store.steampowered.com/app/1158310/Cru...,Crusader Kings III,18951 92 18951,1-Sep-20,System RequirementsWindowsMac OS XSteamOS + Li...
...,...,...,...,...,...
4756,https://store.steampowered.com/app/403640/Dish...,Dishonored 2,20889 85 20889,"11 Nov, 2016",System RequirementsMinimum:Requires a 64-bit p...
4757,https://store.steampowered.com/app/939850/The_...,The Dark Pictures Anthology: Man of Medan,3584 70 3584,"29 Aug, 2019",System RequirementsMinimum:Requires a 64-bit p...
4758,https://store.steampowered.com/app/200710/Torc...,Torchlight II,26709 94 26709,"20 Sep, 2012",System RequirementsWindowsMac OS XSteamOS + Li...
4759,https://store.steampowered.com/app/461430/Emer...,EmergeNYC,1982 65 1982,"3 Nov, 2016",System RequirementsWindowsMac OS XMinimum:Requ...


With the useless information gone, we can get the number of reviews and percetange of likes.

In [17]:
num_reviews = []
pct_like = []

for x in range(0, df_game.shape[0]):
    review = df_game.loc[x, "all_reviews"].split()
    
    num_reviews.append(int(review[0]))
    pct_like.append(int(review[1]))
    
print(len(num_reviews), len(pct_like))

4761 4761


Adding number of reviews and percentage of likes columns. Also getting rid of the "all_reviews" column

In [18]:
df_game["num_reviews"] = num_reviews
df_game["pct_like"] = pct_like
df_game.drop(columns="all_reviews", inplace=True)
df_game

<ipython-input-18-fa3e47b5cbce>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_game["num_reviews"] = num_reviews
<ipython-input-18-fa3e47b5cbce>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_game["pct_like"] = pct_like
C:\Users\wilso\anaconda3\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,url,name,date,requirements,num_reviews,pct_like
0,https://store.steampowered.com/app/945360/Amon...,Among Us,16-Nov-18,System RequirementsMinimum:OS: Windows 7 SP1+P...,224878,95
1,https://store.steampowered.com/app/730/Counter...,Counter-Strike: Global Offensive,21-Aug-12,System RequirementsWindowsMac OS XSteamOS + Li...,4843904,87
2,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,3-Aug-20,System RequirementsMinimum:Requires a 64-bit p...,223706,80
3,https://store.steampowered.com/app/1097150/Fal...,Fall Guys: Ultimate Knockout,3-Aug-20,System RequirementsMinimum:Requires a 64-bit p...,223725,80
4,https://store.steampowered.com/app/1158310/Cru...,Crusader Kings III,1-Sep-20,System RequirementsWindowsMac OS XSteamOS + Li...,18951,92
...,...,...,...,...,...,...
4756,https://store.steampowered.com/app/403640/Dish...,Dishonored 2,"11 Nov, 2016",System RequirementsMinimum:Requires a 64-bit p...,20889,85
4757,https://store.steampowered.com/app/939850/The_...,The Dark Pictures Anthology: Man of Medan,"29 Aug, 2019",System RequirementsMinimum:Requires a 64-bit p...,3584,70
4758,https://store.steampowered.com/app/200710/Torc...,Torchlight II,"20 Sep, 2012",System RequirementsWindowsMac OS XSteamOS + Li...,26709,94
4759,https://store.steampowered.com/app/461430/Emer...,EmergeNYC,"3 Nov, 2016",System RequirementsWindowsMac OS XMinimum:Requ...,1982,65


Reformatting the date

Since the dates have different formats, we need a way to standardize them. In this case, we can write our own function to do this. Days and months will be rewritten as two digits numbers, years will be rewritten as four. 

For the past two years or so (2020, 2021), there has been a chip shortage. To combat this, games from these two years will not be included. Also, DirectX 12 came out in 2015, so games before this will not be included as well.

In [19]:
def standardize_date(str_in):
    str_in = str_in.lower()
    str_in = str_in.strip()
    
    str_in = str_in.replace(",", "")
    str_in = str_in.replace("-", " ")
    
    s = str_in.split()
    
    try:
        if (s[0] in ["1", "2", "3", "4", "5", "6", "7", "8", "9"]):
            s[0] = f"0{s[0]}"

        s[1] = s[1].replace("jan", "01")
        s[1] = s[1].replace("feb", "02")
        s[1] = s[1].replace("mar", "03")
        s[1] = s[1].replace("apr", "04")
        s[1] = s[1].replace("may", "05")
        s[1] = s[1].replace("jun", "06")
        s[1] = s[1].replace("jul", "07")
        s[1] = s[1].replace("aug", "08")
        s[1] = s[1].replace("sep", "09")
        s[1] = s[1].replace("oct", "10")
        s[1] = s[1].replace("nov", "11")
        s[1] = s[1].replace("dec", "12")
        
        if (s[2] not in ["15", "16", "17", "18", "19"]):
            return 0

        s[2] = s[2].replace("2015", "15")
        s[2] = s[2].replace("2016", "16")
        s[2] = s[2].replace("2017", "17")
        s[2] = s[2].replace("2018", "18")
        s[2] = s[2].replace("2019", "19")
        
        s[2] = s[2].replace("15", "2015")
        s[2] = s[2].replace("16", "2016")
        s[2] = s[2].replace("17", "2017")
        s[2] = s[2].replace("18", "2018")
        s[2] = s[2].replace("19", "2019")
    except:
        return 0
    
    str_in = f"{s[0]}/{s[1]}/{s[2]}"
    
    return str_in

df_game["date"] = df_game["date"].apply(standardize_date)
df_game["date"]

<ipython-input-19-a2db851eb8cd>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_game["date"] = df_game["date"].apply(standardize_date)


0       16/11/2018
1                0
2                0
3                0
4                0
           ...    
4756             0
4757             0
4758             0
4759             0
4760             0
Name: date, Length: 4761, dtype: object

Get rid of games not within 2015-2019

In [20]:
df_game = df_game[df_game["date"] != 0]
df_game

,url,name,date,requirements,num_reviews,pct_like
0,https://store.steampowered.com/app/945360/Amon...,Among Us,16/11/2018,System RequirementsMinimum:OS: Windows 7 SP1+P...,224878,95
5,https://store.steampowered.com/app/1085660/Des...,Destiny 2,01/10/2019,System RequirementsMinimum:Requires a 64-bit p...,284689,86
8,https://store.steampowered.com/app/359550/Tom_...,Tom Clancy's Rainbow Six® Siege,01/12/2015,"System RequirementsMinimum:OS: Windows 7, Wind...",642433,88
9,https://store.steampowered.com/app/359550/Tom_...,Tom Clancy's Rainbow Six® Siege,01/12/2015,"System RequirementsMinimum:OS: Windows 7, Wind...",642418,88
17,https://store.steampowered.com/app/381210/Dead...,Dead by Daylight,14/06/2016,System RequirementsMinimum:Requires a 64-bit p...,274935,82
...,...,...,...,...,...,...
4357,https://store.steampowered.com/app/369720/Moon...,Moonrise Base Game + Guildmaster's Edition,27/05/2015,System RequirementsWindowsMac OS XMinimum:OS: ...,32,31
4358,https://store.steampowered.com/app/369720/Moon...,Moonrise Base Game + Guildmaster's Edition,27/05/2015,System RequirementsWindowsMac OS XMinimum:OS: ...,32,31
4359,https://store.steampowered.com/app/369720/Moon...,Moonrise Base Game + Guildmaster's Edition,27/05/2015,System RequirementsWindowsMac OS XMinimum:OS: ...,32,31
4360,https://store.steampowered.com/app/755770/YouT...,YouTube VR,14/12/2017,System RequirementsMinimum:Requires a 64-bit p...,1086,19


Some games were duplicated and not removed earlier. Removing them now

In [21]:
df_game.sort_values(by=["num_reviews"], ascending=False, inplace=True)
df_game.drop_duplicates(subset="name", inplace=True)
df_game.set_index(np.array(range(0, df_game.shape[0])), inplace=True)
df_game

<ipython-input-21-2210a1ff87c3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_game.sort_values(by=["num_reviews"], ascending=False, inplace=True)
<ipython-input-21-2210a1ff87c3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_game.drop_duplicates(subset="name", inplace=True)


,url,name,date,requirements,num_reviews,pct_like
0,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,21/12/2017,System RequirementsMinimum:Requires a 64-bit p...,1316559,52
1,https://store.steampowered.com/app/271590/Gran...,Grand Theft Auto V,14/04/2015,System RequirementsMinimum:Requires a 64-bit p...,841021,80
2,https://store.steampowered.com/app/359550/Tom_...,Tom Clancy's Rainbow Six® Siege,01/12/2015,"System RequirementsMinimum:OS: Windows 7, Wind...",642433,88
3,https://store.steampowered.com/app/304930/Untu...,Unturned,07/07/2017,System RequirementsWindowsMac OS XSteamOS + Li...,402298,91
4,https://store.steampowered.com/app/292030/The_...,The Witcher® 3: Wild Hunt,18/05/2015,System RequirementsMinimum:OS: 64-bit Windows ...,386212,98
...,...,...,...,...,...,...
2475,https://store.steampowered.com/app/1018130/Cas...,Castle Break,19/02/2019,System RequirementsMinimum:OS: Windows 7 or ne...,45,95
2476,https://store.steampowered.com/app/763710/Rive...,River City Melee Mach!!,10/10/2019,System RequirementsMinimum:Requires a 64-bit p...,41,85
2477,https://store.steampowered.com/app/705210/Cube...,Cube Racer,30/10/2017,System RequirementsWindowsSteamOS + LinuxMinim...,37,89
2478,https://store.steampowered.com/app/369720/Moon...,Moonrise Base Game + Guildmaster's Edition,27/05/2015,System RequirementsWindowsMac OS XMinimum:OS: ...,32,31


Web scraping for requirements

Some of the requirements are missing. However, since we have the urls for the games, we can easily web scrape for them instead. This processes takes a while (20 - 30 minutes)

In [22]:
min_cards = []
rec_cards = []
minimum = ""
recommended = ""
contains_graphics_section = False

for index in range(0, df_game.shape[0]):
    try:
        URL = df_game.loc[index, "url"]
        page = requests.get(URL)

        soup = BeautifulSoup(page.content, "html.parser")

        card = soup.find("div", {"class": "game_area_sys_req sysreq_content active"})
        card = card.find_all("ul", {"class":"bb_ul"})

        for x in range(0, len(card)):
            contains_graphics_section = False
            rec = card[x].find_all("li")
            for tag in rec:
                if ("Graphics" in tag.text.strip()):
                    contains_graphics_section = True
                    if (x == 0):
                        minimum = tag.text.strip()
                        recommended = tag.text.strip()
                    elif (x == 1):
                        recommended = tag.text.strip()

        if (not contains_graphics_section):
            min_cards.append("NA")
            rec_cards.append("NA")
        else:
            min_cards.append(minimum)
            rec_cards.append(recommended)

        page.close()
    except:
        min_cards.append("NA")
        rec_cards.append("NA")
                
print(len(min_cards), len(rec_cards))

2480 2480


Now we can add columns for minimum and recommended hardware.

Some of these games may not be on Steam anymore and the GPU requirements can no longer be scraped. In this case, we can just remove them from our dataset.

In [27]:
df_game["minimum"] = min_cards
df_game["recommended"] = rec_cards
df_game = df_game.drop(labels="requirements", axis=1)
df_game = df_game[df_game["minimum requirement"] != "NA"]

Now we have

In [30]:
df_game

,url,name,date,num_reviews,pct_like,minimum,recommended
0,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,21/12/2017,1316559,52,Graphics: NVIDIA GeForce GTX 960 2GB / AMD Rad...,Graphics: NVIDIA GeForce GTX 1060 3GB / AMD Ra...
1,https://store.steampowered.com/app/271590/Gran...,Grand Theft Auto V,14/04/2015,841021,80,Graphics: NVIDIA 9800 GT 1GB / AMD HD 4870 1GB...,Graphics: NVIDIA GTX 660 2GB / AMD HD 7870 2GB
2,https://store.steampowered.com/app/359550/Tom_...,Tom Clancy's Rainbow Six® Siege,01/12/2015,642433,88,Graphics: NVIDIA GeForce GTX 460 or AMD Radeon...,Graphics: NVIDIA GeForce GTX 670 (or GTX 760 /...
4,https://store.steampowered.com/app/292030/The_...,The Witcher® 3: Wild Hunt,18/05/2015,386212,98,Graphics: Nvidia GPU GeForce GTX 660 / AMD GPU...,Graphics: Nvidia GPU GeForce GTX 770 / AMD GPU...
5,https://store.steampowered.com/app/252490/Rust...,Rust,08/02/2018,375523,84,Graphics: GTX 670 2GB / AMD R9 280 better,Graphics: GTX 980 / AMD R9 Fury
...,...,...,...,...,...,...,...
2472,https://store.steampowered.com/app/439800/Clou...,Clouds & Sheep 2,24/10/2016,59,83,Graphics: 1 GB,Graphics: 1 GB
2474,https://store.steampowered.com/app/1172520/Col...,Colorgrid,21/11/2019,50,100,Graphics: Graphics card supporting DirectX 9.0c,Graphics: Graphics card supporting DirectX 9.0c
2475,https://store.steampowered.com/app/1018130/Cas...,Castle Break,19/02/2019,45,95,Graphics: existing,Graphics: working with 1920x1080
2477,https://store.steampowered.com/app/705210/Cube...,Cube Racer,30/10/2017,37,89,Graphics: 512 MB,Graphics: 1 GB


Remove "Graphics:" from the added columns

In [39]:
df_game["minimum"] = df_game["minimum"].apply(lambda x: (x.replace("Graphics:", "")).strip())
df_game["recommended"] = df_game["recommended"].apply(lambda x: (x.replace("Graphics:", "")).strip())

And we get

In [40]:
df_game

,url,name,date,num_reviews,pct_like,minimum,recommended
0,https://store.steampowered.com/app/578080/PLAY...,PLAYERUNKNOWN'S BATTLEGROUNDS,21/12/2017,1316559,52,NVIDIA GeForce GTX 960 2GB / AMD Radeon R7 370...,NVIDIA GeForce GTX 1060 3GB / AMD Radeon RX 58...
1,https://store.steampowered.com/app/271590/Gran...,Grand Theft Auto V,14/04/2015,841021,80,"NVIDIA 9800 GT 1GB / AMD HD 4870 1GB (DX 10, 1...",NVIDIA GTX 660 2GB / AMD HD 7870 2GB
2,https://store.steampowered.com/app/359550/Tom_...,Tom Clancy's Rainbow Six® Siege,01/12/2015,642433,88,NVIDIA GeForce GTX 460 or AMD Radeon HD 5870 (...,NVIDIA GeForce GTX 670 (or GTX 760 / GTX 960) ...
4,https://store.steampowered.com/app/292030/The_...,The Witcher® 3: Wild Hunt,18/05/2015,386212,98,Nvidia GPU GeForce GTX 660 / AMD GPU Radeon HD...,Nvidia GPU GeForce GTX 770 / AMD GPU Radeon R9...
5,https://store.steampowered.com/app/252490/Rust...,Rust,08/02/2018,375523,84,GTX 670 2GB / AMD R9 280 better,GTX 980 / AMD R9 Fury
...,...,...,...,...,...,...,...
2472,https://store.steampowered.com/app/439800/Clou...,Clouds & Sheep 2,24/10/2016,59,83,1 GB,1 GB
2474,https://store.steampowered.com/app/1172520/Col...,Colorgrid,21/11/2019,50,100,Graphics card supporting DirectX 9.0c,card supporting DirectX 9.0c
2475,https://store.steampowered.com/app/1018130/Cas...,Castle Break,19/02/2019,45,95,existing,working with 1920x1080
2477,https://store.steampowered.com/app/705210/Cube...,Cube Racer,30/10/2017,37,89,512 MB,1 GB
